# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("weatherData.csv")
df.head()


,Cities,Latitude,Longitude,Temperature,Humidity,Pressure,Wind_speed,Cloudiness
0,yellowknife,62.46,-114.35,57.52,100,1010,8.05,90
1,pisco,-13.70,-76.22,55.06,93,1015,4.70,75
2,guerrero negro,27.98,-114.06,68.29,77,1014,11.72,0
3,victoria,22.29,114.16,88.65,66,1005,10.83,100
4,makakilo city,21.35,-158.09,77.79,69,1015,5.82,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
Humid = df["Humidity"]


# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
mask1 = (df.Temperature >= 70) & (df.Temperature < 80)
mask2 = df.Wind_speed < 10
mask3 = df.Cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
ideal_cities

,Cities,Latitude,Longitude,Temperature,Humidity,Pressure,Wind_speed,Cloudiness
0,henties bay,-22.12,14.28,72.00,26,1019,1.99,0
1,conde,-7.26,-34.91,70.66,92,1014,9.10,0
2,odzun,41.05,44.61,72.37,58,1014,4.43,0
3,vikulovo,56.82,70.61,73.65,46,1013,9.22,0
4,jaru,-10.44,-62.47,71.29,77,1010,1.57,0
5,gat,31.61,34.76,79.00,75,1010,6.35,0
6,latina,41.45,13.10,74.43,88,1006,5.01,0
7,lazaro cardenas,17.96,-102.20,78.13,77,1014,2.28,0
8,zakharovo,54.37,39.28,77.61,75,1013,7.11,0
9,balsas,-7.53,-46.04,71.46,54,1012,2.59,0


In [15]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["Latitude", "Longitude"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [19]:
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.Cities}</dd>
                <dt>Temp</dt><dd>{row.Temperature}</dd>
                <dt>Cloudiness</dt><dd>{row.Cloudiness}%</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [21]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
ideal_cities.head(1)

,Cities,Latitude,Longitude,Temperature,Humidity,Pressure,Wind_speed,Cloudiness
0,henties bay,-22.12,14.28,72.0,26,1019,1.99,0


In [36]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

lat = ideal_cities["Latitude"][1]
lng = ideal_cities["Longitude"][1]

params["Location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)
name_address = name_address.json()

#name_address['results'][0]['name']

In [25]:

#loop locations
names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # create url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("No hotels listed nearby.")
        names.append("No listed hotels")
        addresses.append("")
        ratings.append("")

No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.


In [33]:
ideal_cities["Hotel"] = names
ideal_cities["Address"] = addresses
ideal_cities["Rating"] = ratings
ideal_cities.head(15)

,Cities,Latitude,Longitude,Temperature,Humidity,Pressure,Wind_speed,Cloudiness,Hotel,Address,Rating
0,henties bay,-22.12,14.28,72.00,26,1019,1.99,0,No listed hotels,,
1,conde,-7.26,-34.91,70.66,92,1014,9.10,0,No listed hotels,,
2,odzun,41.05,44.61,72.37,58,1014,4.43,0,No listed hotels,,
3,vikulovo,56.82,70.61,73.65,46,1013,9.22,0,No listed hotels,,
4,jaru,-10.44,-62.47,71.29,77,1010,1.57,0,No listed hotels,,
5,gat,31.61,34.76,79.00,75,1010,6.35,0,No listed hotels,,
6,latina,41.45,13.10,74.43,88,1006,5.01,0,No listed hotels,,
7,lazaro cardenas,17.96,-102.20,78.13,77,1014,2.28,0,No listed hotels,,
8,zakharovo,54.37,39.28,77.61,75,1013,7.11,0,No listed hotels,,
9,balsas,-7.53,-46.04,71.46,54,1012,2.59,0,No listed hotels,,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
